# Prepare Data

## 1. Extract frames using FaceForecnsics++ repo

In [1]:
# Original command

import os
from os.path import join
import argparse
import subprocess
import cv2
from tqdm import tqdm


DATASET_PATHS = {
    'original': 'original_sequences',
    'Deepfakes': 'manipulated_sequences/Deepfakes',
    'Face2Face': 'manipulated_sequences/Face2Face',
    'FaceSwap': 'manipulated_sequences/FaceSwap'
}
COMPRESSION = ['c0', 'c23', 'c40']


def extract_frames(data_path, output_path, method='cv2'):
    """Method to extract frames, either with ffmpeg or opencv. FFmpeg won't
    start from 0 so we would have to rename if we want to keep the filenames
    coherent."""
    os.makedirs(output_path, exist_ok=True)
    if method == 'ffmpeg':
        subprocess.check_output(
            'ffmpeg -i {} {}'.format(
                data_path, join(output_path, '%04d.png')),
            shell=True, stderr=subprocess.STDOUT)
    elif method == 'cv2':
        reader = cv2.VideoCapture(data_path)
        frame_num = 0
        while reader.isOpened():
            success, image = reader.read()
            if not success:
                break
            cv2.imwrite(join(output_path, '{:04d}.png'.format(frame_num)),
                        image)
            frame_num += 1
        reader.release()
    else:
        raise Exception('Wrong extract frames method: {}'.format(method))


def extract_method_videos(data_path, dataset, compression):
    """Extracts all videos of a specified method and compression in the
    FaceForensics++ file structure"""
    # videos_path = join(data_path, DATASET_PATHS[dataset], compression, 'videos')
    images_path = join(data_path, DATASET_PATHS[dataset], compression, 'images')
    # print("list = ", tqdm(os.listdir(data_path)))

    for video in tqdm(os.listdir(data_path)):
        image_folder = video.split('.')[0]
        extract_frames(join(data_path, video),
                       join(images_path, image_folder))




In [2]:

# Original command
p = argparse.ArgumentParser( formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# p.add_argument('--data_path', type=str)
p.add_argument('--dataset', '-d', type=str,
               choices=list(DATASET_PATHS.keys()) + ['all'],
               default='original')
p.add_argument('--compression', '-c', type=str, choices=COMPRESSION,
               default='c0')

p.add_argument('--data_path', default="examples/")


args = p.parse_args('')  # important to put '' in Jupyter otherwise it will complain
print(args)

extract_method_videos(**vars(args))


Namespace(compression='c0', data_path='examples/', dataset='original')



00%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.30s/it]

## 2. copy file to the right location

In [3]:
import shutil

new_path = shutil.copytree('examples/original_sequences/c0/images/id0_id16_0000', 
                           'data/datasets/CelebDF/FakeCelebDF/images/id0_id16_0000')
new_path = shutil.copytree('examples/original_sequences/c0/images/id0_0000',
                          'data/datasets/CelebDF/RealCelebDF/images/id0_0000')


## 3. Detect face and compute 68 landmarks

In [4]:
import face_alignment
import os
import numpy as np
from skimage import io
paths = ["data/datasets/CelebDF/FakeCelebDF/images/id0_id16_0000",
         "data/datasets/CelebDF/RealCelebDF/images/id0_0000"]

for path in paths: 
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False)
    preds = fa.get_landmarks_from_directory(path)
    # print(path)
    # print(len(preds))
    for k in preds:
        # print(k)
        item = preds[k][0]
        # print(type(item),item)
    
        filename = k.replace("images", "landmarks").replace(".png", ".npy")
        folder = os.path.dirname(filename)
        if not os.path.exists(folder):
            os.makedirs(folder)
    
        np.save(filename, item)  
        # break
        



00%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:21<00:00, 21.44it/s]

In [11]:
# print((preds['data/datasets/Forensics/CelebDF/FakeCelebDF/images/id0_id16_0000/0091.png'])[0].shape)

(68, 2)


## 4. Crop Mouth & Evaluation


Finally run the crop image using

```
python preprocessing/crop_mouths.py --dataset CelebDF --data-root data/datasets/

```



## 5. Evaluation 
a. Download model from  ```https://drive.google.com/file/d/1wfZnxZpyNd5ouJs0LjVls7zU0N_W73L7/view?usp=sharing ```

and copy ```lipforensics_ff.pth``` to main directory

b. Evaluation

the crop-mouths will be store in directory ```cropped_mouths```.

Finally ready for running the evaluation 

```
python evaluate.py --dataset CelebDF --weights_forgery lipforensics_ff.pth
```


